# Validação do Ground Truth Manual

Este notebook valida se cada detecção no ground truth realmente corresponde a um smell válido no código.

**Abordagem:**
1. Carregar ground truth em DataFrame
2. Para cada detecção, ler o arquivo e verificar se o smell está presente na linha especificada
3. Validar conforme os critérios de cada tipo de smell
4. Gerar relatório de validação



In [ ]:
import json
import pandas as pd
import re
import os
from pathlib import Path

BASE_DIR = Path().resolve().parent
GROUND_TRUTH_PATH = BASE_DIR / "dataset" / "ground_truth" / "ground_truth_manual.csv"
DATASET_PATH = BASE_DIR / "dataset"

print(f"Base dir: {BASE_DIR}")
print(f"Ground truth: {GROUND_TRUTH_PATH.exists()}")
print(f"Dataset: {DATASET_PATH.exists()}")



In [ ]:
with open(GROUND_TRUTH_PATH, "r", encoding="utf-8") as f:
    gt_data = json.load(f)

df_gt = pd.DataFrame(gt_data)

print(f"Total de detecções: {len(df_gt)}")
print("\nTipos de smells:")
print(df_gt["Smell"].value_counts())
print("\nPrimeiras linhas:")
df_gt.head()


In [ ]:
# Função para ler arquivo
def read_file_lines(file_path):
    """Lê um arquivo e retorna lista de linhas."""
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            return f.readlines()
    except Exception as e:
        return None


def get_line_content(file_path, line_no):
    """Retorna o conteúdo de uma linha específica."""
    lines = read_file_lines(file_path)
    if not lines:
        return None
    try:
        line_num = int(line_no)
        if 1 <= line_num <= len(lines):
            return lines[line_num - 1].rstrip("\n")
    except:
        pass
    return None


# Teste
test_file = df_gt.iloc[0]["File"]
test_line = df_gt.iloc[0].get("Line no", "")
print(f"Teste - Arquivo: {test_file}")
print(f"Teste - Linha: {test_line}")
if test_line:
    content = get_line_content(test_file, test_line)
    print(f"Teste - Conteúdo: {content}")


In [ ]:
def validate_magic_number(row):
    """Valida Magic Number: verifica se o número está na linha."""
    file_path = row["File"]
    line_no = row.get("Line no", "").strip()
    details = row.get("Details", "")

    if not line_no:
        return {"valid": False, "reason": "Linha não especificada"}

    if not os.path.exists(file_path):
        return {"valid": False, "reason": "Arquivo não encontrado"}

    line_content = get_line_content(file_path, line_no)
    if not line_content:
        return {"valid": False, "reason": f"Linha {line_no} não existe"}

    match = re.search(r"Magic number ([\d.eE+-]+)", details)
    if not match:
        return {"valid": False, "reason": "Não foi possível extrair número do Details"}

    expected_number = match.group(1)

    number_patterns = [
        rf"\b{re.escape(expected_number)}\b",  # Número exato
        rf"{re.escape(expected_number)}",  # Número em qualquer contexto
    ]

    found = False
    for pattern in number_patterns:
        if re.search(pattern, line_content):
            found = True
            break

    if not found:
        return {
            "valid": False,
            "reason": f"Número {expected_number} não encontrado na linha",
        }

    # Verificar se é número trivial (0, 1, -1)
    try:
        num_value = float(expected_number)
        if num_value in [0, 1, -1]:
            return {
                "valid": True,
                "reason": f"Número {expected_number} encontrado (mas é trivial)",
            }
    except:
        pass

    return {"valid": True, "reason": "Número encontrado na linha"}


In [ ]:
def validate_long_message_chain(row):
    """Valida Long Message Chain: verifica se há cadeia de métodos na linha."""
    file_path = row["File"]
    line_no = row.get("Line no", "").strip()
    details = row.get("Details", "")

    if not line_no:
        return {"valid": False, "reason": "Linha não especificada"}

    if not os.path.exists(file_path):
        return {"valid": False, "reason": "Arquivo não encontrado"}

    line_content = get_line_content(file_path, line_no)
    if not line_content:
        return {"valid": False, "reason": f"Linha {line_no} não existe"}

    dots = line_content.count(".")

    if dots < 2:
        return {
            "valid": False,
            "reason": f"Apenas {dots} ponto(s) encontrado(s), esperado >= 2",
        }

    pattern = r"\w+\.\w+\.\w+"
    if not re.search(pattern, line_content):
        return {"valid": False, "reason": "Não parece ser uma cadeia de métodos válida"}

    return {"valid": True, "reason": f"Cadeia com {dots + 1} métodos encontrada"}


In [ ]:
def validate_long_lambda(row):
    """Valida Long Lambda Function: verifica se há lambda > 80 caracteres na linha."""
    file_path = row["File"]
    line_no = row.get("Line no", "").strip()
    details = row.get("Details", "")

    if not line_no:
        return {"valid": False, "reason": "Linha não especificada"}

    if not os.path.exists(file_path):
        return {"valid": False, "reason": "Arquivo não encontrado"}

    line_content = get_line_content(file_path, line_no)
    if not line_content:
        return {"valid": False, "reason": f"Linha {line_no} não existe"}

    # Verificar se há lambda
    if "lambda" not in line_content.lower():
        return {"valid": False, "reason": "Não há lambda na linha"}

    # Encontrar expressão lambda completa
    # Padrão: lambda ... : ...
    lambda_match = re.search(r"lambda[^:]*:[^,)]*", line_content)
    if not lambda_match:
        return {"valid": False, "reason": "Não foi possível extrair expressão lambda"}

    lambda_expr = lambda_match.group(0)
    actual_chars = len(lambda_expr)

    if actual_chars <= 80:
        return {
            "valid": False,
            "reason": f"Lambda tem apenas {actual_chars} caracteres (threshold: 80)",
        }

    return {"valid": True, "reason": f"Lambda com {actual_chars} caracteres encontrada"}


In [ ]:
def find_method_range(file_path, method_name):
    """Encontra início e fim de um método. Retorna (linha_inicio, linha_fim)."""
    lines = read_file_lines(file_path)
    if not lines:
        return None

    start_line = None
    indent_level = None

    for i, line in enumerate(lines, 1):
        if f"def {method_name}(" in line or f"def {method_name}:" in line:
            start_line = i
            indent_level = len(line) - len(line.lstrip())
            break

    if start_line is None:
        return None

    for i in range(start_line, len(lines)):
        line = lines[i]
        if i == start_line:
            continue
        current_indent = len(line) - len(line.lstrip())
        if (
            line.strip()
            and current_indent <= indent_level
            and not line.strip().startswith("#")
        ):
            return (start_line, i - 1)

    return (start_line, len(lines))


def validate_long_method(row):
    """Valida Long Method: verifica se método tem > 67 linhas."""
    file_path = row["File"]
    method = row.get("Method", "").strip()
    details = row.get("Details", "")

    if not method:
        return {"valid": False, "reason": "Método não especificado"}

    if not os.path.exists(file_path):
        return {"valid": False, "reason": "Arquivo não encontrado"}

    method_range = find_method_range(file_path, method)
    if not method_range:
        return {"valid": False, "reason": f'Método "{method}" não encontrado'}

    start_line, end_line = method_range
    actual_lines = end_line - start_line + 1

    if actual_lines <= 67:
        return {
            "valid": False,
            "reason": f"Método tem apenas {actual_lines} linhas (threshold: 67)",
        }

    return {"valid": True, "reason": f"Método com {actual_lines} linhas encontrado"}


In [ ]:
def validate_detection(row):
    """Valida uma detecção baseado no tipo de smell."""
    smell_type = row.get("Smell", "")

    validators = {
        "Magic Number": validate_magic_number,
        "Long Method": validate_long_method,
        "Long Message Chain": validate_long_message_chain,
        "Long Lambda Function": validate_long_lambda,
    }

    validator = validators.get(smell_type)
    if not validator:
        return {
            "valid": None,
            "reason": f"Validador não implementado para: {smell_type}",
        }

    return validator(row)


In [ ]:
print("Validando detecções...")
results = []

for idx, row in df_gt.iterrows():
    result = validate_detection(row)
    results.append(
        {
            "index": idx,
            "File": row["File"],
            "Smell": row["Smell"],
            "Method": row.get("Method", ""),
            "Line no": row.get("Line no", ""),
            "valid": result["valid"],
            "reason": result["reason"],
        }
    )

    if (idx + 1) % 50 == 0:
        print(f"  Processadas: {idx + 1}/{len(df_gt)}")

df_results = pd.DataFrame(results)
print("\nValidação concluída!")


In [ ]:
# Estatísticas gerais
print("=" * 80)
print("ESTATÍSTICAS DE VALIDAÇÃO")
print("=" * 80)

total = len(df_results)
valid = len(df_results[df_results["valid"] == True])
invalid = len(df_results[df_results["valid"] == False])
not_implemented = len(df_results[df_results["valid"].isna()])

print(f"\nTotal de detecções: {total}")
print(f"✓ Válidas: {valid} ({valid / total * 100:.1f}%)")
print(f"❌ Inválidas: {invalid} ({invalid / total * 100:.1f}%)")
print(f"⚠️  Não implementadas: {not_implemented} ({not_implemented / total * 100:.1f}%)")


In [ ]:
print("\n" + "=" * 80)
print("ESTATÍSTICAS POR TIPO DE SMELL")
print("=" * 80)

df_stats = df_results.groupby(["Smell", "valid"]).size().unstack(fill_value=0)
df_stats["Total"] = df_stats.sum(axis=1)
df_stats["% Válidas"] = (df_stats[True] / df_stats["Total"] * 100).round(1)
df_stats = df_stats.sort_values("Total", ascending=False)

print("\n")
print(df_stats)


In [ ]:
print("\n" + "=" * 80)
print("DETECÇÕES INVÁLIDAS (primeiras 20)")
print("=" * 80)

df_invalid = df_results[df_results["valid"] == False].copy()
df_invalid["file_name"] = df_invalid["File"].apply(lambda x: x.split("/")[-1])

print(f"\nTotal de inválidas: {len(df_invalid)}")
print("\nPrimeiras 20:")
for idx, row in df_invalid.head(20).iterrows():
    print(f"\n{row['file_name']} | {row['Smell']} | Linha {row['Line no']}")
    print(f"  Motivo: {row['reason']}")


In [ ]:
print("\n" + "=" * 80)
print("MOTIVOS DE INVALIDAÇÃO")
print("=" * 80)

if len(df_invalid) > 0:
    reasons = df_invalid["reason"].value_counts()
    print("\n")
    for reason, count in reasons.items():
        print(f"{count:3d}x - {reason}")


In [ ]:
output_path = BASE_DIR / "results" / "validation_results.csv"
df_results.to_csv(output_path, index=False, encoding="utf-8")
print(f"\n✓ Resultados salvos em: {output_path}")

print("\n" + "=" * 80)
print("RESUMO FINAL")
print("=" * 80)
print(f"Total validado: {valid + invalid}")
print(f"Taxa de sucesso: {valid / (valid + invalid) * 100:.1f}%")
print(f"Taxa de erro: {invalid / (valid + invalid) * 100:.1f}%")
